# workflows presented in "The long-term instability of glacier dynamics due to a sudden change of basal lubrication"

## (DRAFT) v 0.1

This notebook is not officially published yet. All the content is subject to change at any time. 

### Objectives besides the science

Beside the science, the other important purpose of this notebook is to test and find the best way to present a live workflow along with an academic publication (e.g., a journal paper published by Geophysical Research Letters), maximizing its reproducibility.

We plan to submit this notebook as part of the supporting information of the manuscript (xxx - see the title).

Triditionally, this kind of materials would be zipped and uploaded to the same place where you can find online paper text (probably through a DOI).

In the light of the FAIR data policy, publishers are beginning to ask that the supplemental materials should be easily findable, accessible, and reusable as well. A popular option for geoscience researchers is Zenodo, which we can get a permanent DOI for the data sets, code, and documents uploaded there. 

Here we would like to make one step forward and see how we can include Jupyter notebooks + binder or Jupyter Book pages as the supplemental materials while retaining their functionaility to the users. (i.e. they should NOT be only uploaded to Zenodo as a zipped file).

### Color codes for the challenges 

<font color='blue'> This means a minor issue -- can be resolved without much effort.</font>

<font color='red'> This means a major issue -- needs attention of more people.</font>

## 1. Results from the data of the Greenland Ice Sheet

The notebook calculates $P_e$ and $J_0$ of multiple glaciers in Greenland using the flowline location provided by Felikson et al (2021). <font color='blue'>(Needs a formal ciation and reference section)</font>

The $P_e$ and $J_0$ are then compared with the glacier speed change during 1998-2018 from the ITS_LIVE data set.

<font color='red'> Suppose we get a DOI from Zenodo's service by uploading the supplemental materials. How to make the DOI link to a binder or a Jupyter book page so users can directly execute these cells? Should we contact Zenodo team for more discussion? </font>

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

`pjgris.py` contains few functions for running this notebook. It is not meant to be published as a Python package (at least this is what I think). <font color='blue'>What is a good way to host this file?</font>

In [2]:
from pjgris import my_savgol_filter, savgol_smoothing, pe_corefun
import glob
import rasterio
import utils
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy import interpolate

This notebook needs to import lots of data from the SI of Felikson et al.'s paper. Each `.nc` file here is roughly 20-30 MB. <font color='red'>What is a good way to host these data?</font>

In [3]:
netcdf_dir = '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs'
glaciers = [i for i in glob.glob(netcdf_dir + '/glacier*.nc')]
glaciers.sort()
print('glaciers total: {}'.format(len(glaciers)))
glaciers

glaciers total: 187


['/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0001.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0002.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0003.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0004.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0005.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0007.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0008.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0009.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0010.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0012.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0013.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0014.nc',
 '/home/jovyan/Projects/LubriSens/Data/Felikson2021/netcdfs/glacier0015.nc',

This notebook also uses ITS_LIVE dataset as the input of glacier speed. <font color='blue'>Should we show how to get the data from the ITS_LIVE website, or just give readers the downloaded file?</font>

In [4]:
speed_file = '/home/jovyan/Projects/LubriSens/Data/ITSLIVE/GRE_G0240_1998_v.tif'
speed_data = rasterio.open(speed_file)
vdiff_file = '/home/jovyan/Projects/LubriSens/Data/ITSLIVE/GRE_G0240_diff-2018-1998_v.tif'
vdiff_data = rasterio.open(vdiff_file)


Here comes a long cell because most lines here are within a for loop. <font color='red'> Is it possible to run a for loop across multiple cells so we can make better comments/explanations using Markdown for the code within  the loop? </font>

In [5]:
results = {}

for glacier_file in glaciers:
    ds = Dataset(glacier_file, 'r')
    flowline_groups, _ = utils.get_flowline_groups(ds)
    flowline_group = flowline_groups[3]   # The fourth flowline
    
    x = flowline_group['x'][:]
    y = flowline_group['y'][:]
    d = flowline_group['d'][:]
    b = flowline_group['geometry']['bed']['BedMachine']['nominal']['h'][:]
    s = flowline_group['geometry']['surface']['GIMP']['nominal']['h'][:]
    # pe_felikson = flowline_group['Pe']['GIMP']['nominal'][:]
    
    if d.size < 280:
        continue   # skip really short glacier flowline

    xytuple = [(m, n) for m, n in zip(x, y)]
    sample_gen = speed_data.sample(xytuple)
    u = np.array([float(record) for record in sample_gen])
    u[u < 0] = np.nan
    
    if sum(~np.isnan(u)) <= 20:
        continue
    
    valid_u_d = d[~np.isnan(u)]
    valid_u_u = u[~np.isnan(u)]
    f = interpolate.interp1d(valid_u_d, valid_u_u, bounds_error=False, fill_value=np.nan)
    u_holefilled = f(d.data)
    
    valid_idx = ~np.isnan(u_holefilled)

    x_valid = x[valid_idx]
    y_valid = y[valid_idx]
    d_valid = d[valid_idx]
    s_valid = s[valid_idx]
    b_valid = b[valid_idx]
    u_valid = u_holefilled[valid_idx]

    if s_valid.size < 280:
        continue   # skip really short glacier flowline
    
    # the point closet to the divide = 0 km
    x_valid = np.flip(x_valid)
    y_valid = np.flip(y_valid)
    s_valid = np.flip(s_valid)
    b_valid = np.flip(b_valid)
    u_valid = np.flip(u_valid)
    
    s_sm, b_sm, u_sm, h_sm, dudx_sm, dhdx_sm, slope_sm, d2hdx2_sm, dalphadx_sm = savgol_smoothing(u_valid, s_valid, b_valid, w=251)
    
    pe, j0, term1, term2, term3, term4 = pe_corefun(u_sm, h_sm, dudx_sm, dhdx_sm, slope_sm, dalphadx_sm)
    
   
    xytuple2 = [(m, n) for m, n in zip(x_valid, y_valid)]
    sample_gen2 = vdiff_data.sample(xytuple2)
    udiff = np.array([float(record) for record in sample_gen2])
    udiff[udiff < -6000] = np.nan
    udiff_sm = my_savgol_filter(udiff, window_length=201, polyorder=1, deriv=0, delta=50, mode='interp')
    
    if sum(~np.isnan(udiff_sm)) == 0:
        continue
        
    d_valid_km = d_valid / 1000
    pe *= 10000
    
    # flip again so that d = 0 km indicates front and points upstream
    pe = np.flip(pe)
    j0 = np.flip(j0)
    term1 = np.flip(term1)
    term2 = np.flip(term2)
    term3 = np.flip(term3)
    term4 = np.flip(term4)
    udiff = np.flip(udiff)
    udiff_sm = np.flip(udiff_sm)
    
    data_group = {'d': d_valid_km, 'pe': pe, 'j0': j0, 'term1': term1, 'term2': term2, 'term3': term3, 'term4': term4,  'udiff': udiff, 'udiff_sm': udiff_sm,}
    results[glacier_file.split('/')[-1]] = data_group

In [6]:
u_valid

array([ 219.38729858,  219.38729858,  219.38729858, ..., 1664.50598145,
       1664.50598145, 1664.50598145])

The following cells visualize the results. <font color='blue'>I don't know if making either (or all) of them interactive is a good idea or not.... Maybe we will see some time while working on the other issues.</font>

In [7]:
burd = cm.get_cmap('turbo', 51)
print(burd(1))

(0.21291, 0.12947, 0.37314, 1.0)


In [8]:
# results

fig, ax1 = plt.subplots(2, 1, sharex=True)
for key in results:
    z_max = np.nanmax(results[key]['udiff_sm'])
    z_min = np.nanmin(results[key]['udiff_sm'])
    z_value = z_max if abs(z_max) > abs(z_min) else z_min
    if z_value > 500:
        linecolor = 'xkcd:red'
    elif z_value < -250: 
        linecolor = 'xkcd:blue'
    else:
        linecolor = 'xkcd:green'
    # linecolor = (z_value + 500) / 3500
    # linecolor = 1 if linecolor > 1 else linecolor 
    # linecolor = 0 if linecolor < 0 else linecolor 
    # print(key, linecolor)
    ax1[0].plot(results[key]['d'], results[key]['pe'], color=linecolor)    # color=burd(linecolor)
    ax1[0].set_ylim([-10, 10])
    ax1[1].plot(results[key]['d'], results[key]['udiff_sm'], color=linecolor)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, ax2 = plt.subplots(1, 1, sharex=True)
for key in results:
    z_max = np.nanmax(results[key]['udiff_sm'])
    z_min = np.nanmin(results[key]['udiff_sm'])
    z_value = z_max if abs(z_max) > abs(z_min) else z_min
    if z_value > 500:
        linecolor = 'xkcd:red'
    elif z_value < -250: 
        linecolor = 'xkcd:blue'
    else:
        linecolor = 'xkcd:green'
    # linecolor = (z_value + 500) / 3500
    # linecolor = 1 if linecolor > 1 else linecolor 
    # linecolor = 0 if linecolor < 0 else linecolor 
    ax2.plot(results[key]['pe'][:100], results[key]['j0'][:100], color=linecolor)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# for i, j in zip(results['glacier0021.nc']['d'], results['glacier0021.nc']['pe']):
#     print(i, j)

In [11]:
fig, ax3 = plt.subplots(1, 1, sharex=True)
for key in results:
    z_max = np.nanmax(results[key]['udiff_sm'])
    z_min = np.nanmin(results[key]['udiff_sm'])
    z_value = z_max if abs(z_max) > abs(z_min) else z_min
    if z_value > 500:
        linecolor = 'xkcd:red'
    elif z_value < -250: 
        linecolor = 'xkcd:brown'
    else:
        linecolor = 'xkcd:blue'
    # linecolor = (z_value + 500) / 3500
    # linecolor = 1 if linecolor > 1 else linecolor 
    # linecolor = 0 if linecolor < 0 else linecolor 
    # ax3.plot(results[key]['term1'][:150] + results[key]['term2'][:150] + results[key]['term3'][:150] + results[key]['term4'][:150], results[key]['j0'][:150], color=linecolor)
    ax3.plot(results[key]['term1'][:100] + results[key]['term2'][:100] + results[key]['term3'][:100], results[key]['j0'][:100], color=linecolor)

    
ax3.set_xlabel(r'$\frac{P_e}{\ell}$ (1/m)')
ax3.set_ylabel(r'$J_0$ (m/yr)')
ax3.set_title('Greenland Ice Sheet \n Accelerated glaciers = red; Stable glaciers = blue; Decce. gl. = brown')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Greenland Ice Sheet \n Accelerated glaciers = red; Stable glaciers = blue; Decce. gl. = brown')

In [12]:
# fig, ax4 = plt.subplots(1, 1, sharex=True)
# for key in results:
#     z_max = np.nanmax(results[key]['udiff_sm'])
#     z_min = np.nanmin(results[key]['udiff_sm'])
#     z_value = z_max if abs(z_max) > abs(z_min) else z_min
#     if z_value > 500:
#         linecolor = 'xkcd:red'
#     elif z_value < -250: 
#         linecolor = 'xkcd:brown'
#     else:
#         linecolor = 'xkcd:blue'
#     # linecolor = (z_value + 500) / 3500
#     # linecolor = 1 if linecolor > 1 else linecolor 
#     # linecolor = 0 if linecolor < 0 else linecolor 
#     # ax3.plot(results[key]['term1'][:150] + results[key]['term2'][:150] + results[key]['term3'][:150] + results[key]['term4'][:150], results[key]['j0'][:150], color=burd(linecolor))
#     ax4.plot(results[key]['term1'][:100] + results[key]['term2'][:100] + results[key]['term3'][:100], results[key]['j0'][:100], color=linecolor)

    
# ax4.set_xlabel(r'$\frac{P_e}{\ell}$ (1/m)')
# ax4.set_ylabel(r'$J_0$ (m/yr)')
# ax4.set_title('Greenland Ice Sheet \n Accelerated glaciers = red; Stable glaciers = blue; Decce. gl. = brown')

In [13]:
from pathlib import Path
Path(glacier_file).stem

'glacierd216'